# Trees and Graphs

In [10]:
class Graph:
    def __init__(self):
        self.nodes = []
    def __str__(self):
        return "\n".join([f"{node}: {[friend.name for friend in node.friends]}" for node in self.nodes])
        # return "".join([f"{f}: {f.friends}" for f in self.nodes])

class Node:
    def __init__(self, name):
        self.friends = []
        self.name = name
    def __str__(self):
        return self.name

s = Node("s")
p = Node("p")
m = Node("m")
l = Node("l")
r = Node("r")
e = Node("e")

s.friends = [p, m]
m.friends = [l]
r.friends = [e]
g = Graph()
g.nodes = [s, p, m, l, r, e]
print(g)

s: ['p', 'm']
p: []
m: ['l']
l: []
r: ['e']
e: []


## Route between nodes

In [12]:
from collections import deque

def isThereARoute(n1, n2):
    # do BFS from n1 to find n2
    q = deque()
    marked = set()
    q.append(n1)
    while q:
        n = q.popleft()
        if n == n2:
            return True
        else:
            if n in marked:
                pass
            else:
                marked.add(n)
                for friend in n.friends:
                    q.append(friend)
    return False
print(isThereARoute(s, e))

False
